In [0]:
# !pip install -q tensorflow-gpu==2.0.0-rc1

In [0]:
# !pip install tf-nightly-gpu

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf

In [0]:
class SEBlock(tf.keras.layers.Layer):
    def __init__(self, input_channels, r=16):
        super(SEBlock, self).__init__()
        self.pool = tf.keras.layers.GlobalAveragePooling2D()
        self.fc1 = tf.keras.layers.Dense(units=input_channels // r)
        self.fc2 = tf.keras.layers.Dense(units=input_channels)

    def call(self, inputs, **kwargs):
        branch = self.pool(inputs)
        branch = self.fc1(branch)
        branch = tf.nn.relu(branch)
        branch = self.fc2(branch)
        branch = tf.nn.sigmoid(branch)
        branch = tf.expand_dims(input=branch, axis=1)
        branch = tf.expand_dims(input=branch, axis=1)
        output = tf.keras.layers.multiply(inputs=[inputs, branch])
        return output


l2 = tf.keras.regularizers.l2

class ConvBlock(tf.keras.Model):
  def __init__(self, num_filters, data_format, weight_decay=1e-4,
               dropout_rate=0, use_se = False):
    super(ConvBlock, self).__init__()

    axis = -1 if data_format == "channels_last" else 1
    inter_filter = num_filters * 4
    # don't forget to set use_bias=False when using batchnorm
    self.conv2 = tf.keras.layers.Conv2D(num_filters,
                                        (3, 3),
                                        padding="same",
                                        use_bias=False,
                                        data_format=data_format,
                                        kernel_initializer="he_normal",
                                        kernel_regularizer=l2(weight_decay))
    self.batchnorm1 = tf.keras.layers.BatchNormalization(axis=axis)
    self.dropout = tf.keras.layers.Dropout(dropout_rate)

    self.conv1 = tf.keras.layers.Conv2D(inter_filter,
                                        (1, 1),
                                        padding="same",
                                        use_bias=False,
                                        data_format=data_format,
                                        kernel_initializer="he_normal",
                                        kernel_regularizer=l2(weight_decay))
    self.batchnorm2 = tf.keras.layers.BatchNormalization(axis=axis)

    self.use_se = use_se
    self.se_block = SEBlock(input_channels=inter_filter)

  def call(self, x, training=True):
    output = self.batchnorm1(x, training=training)

    output = self.conv1(tf.nn.relu(output))
    output = self.batchnorm2(output, training=training)

    output = self.conv2(tf.nn.relu(output))
    output = self.dropout(output, training=training)

    if self.use_se == True:
        x = self.se_block(x)

    return output


class TransitionBlock(tf.keras.Model):
  def __init__(self, num_filters, data_format,
               weight_decay=1e-4, dropout_rate=0):
    super(TransitionBlock, self).__init__()
    axis = -1 if data_format == "channels_last" else 1

    self.batchnorm = tf.keras.layers.BatchNormalization(axis=axis)
    self.conv = tf.keras.layers.Conv2D(num_filters,
                                       (1, 1),
                                       padding="same",
                                       use_bias=False,
                                       data_format=data_format,
                                       kernel_initializer="he_normal",
                                       kernel_regularizer=l2(weight_decay))
    self.avg_pool = tf.keras.layers.AveragePooling2D(data_format=data_format)

  def call(self, x, training=True):
    output = self.batchnorm(x, training=training)
    output = self.conv(tf.nn.relu(output))
    output = self.avg_pool(output)
    return output


class DenseBlock(tf.keras.Model):
  def __init__(self, num_layers, growth_rate, data_format,
               weight_decay=1e-4, dropout_rate=0, use_se=False):
    super(DenseBlock, self).__init__()
    self.num_layers = num_layers
    self.axis = -1 if data_format == "channels_last" else 1

    self.blocks = []
    for _ in range(int(self.num_layers)):
      self.blocks.append(ConvBlock(growth_rate,
                                   data_format,
                                   weight_decay,
                                   dropout_rate,
                                   use_se))

  def call(self, x, training=True):
    for i in range(int(self.num_layers)):
      output = self.blocks[i](x, training=training)
      x = tf.concat([x, output], axis=self.axis)

    return x


class DenseNet(tf.keras.Model):
  def __init__(self, growth_rate, output_classes, num_of_blocks=None, 
               num_layers_in_each_block=None, data_format="channels_last", 
               compression=0.5, weight_decay=1e-4, dropout_rate=0., 
               include_top=True, use_se=False):
    super(DenseNet, self).__init__() 
    self.growth_rate = growth_rate
    self.num_of_blocks = num_of_blocks
    self.output_classes = output_classes
    self.num_layers_in_each_block = num_layers_in_each_block
    self.data_format = data_format
    self.compression = compression
    self.weight_decay = weight_decay
    self.dropout_rate = dropout_rate
    self.include_top = include_top
    self.use_se = use_se

    axis = -1 if self.data_format == "channels_last" else 1

    self.num_filters = 2 * self.growth_rate

    # first conv and pool layer
    self.conv1 = tf.keras.layers.Conv2D(self.num_filters,
                                        (7, 7),
                                        strides=(2, 2),
                                        padding="same",
                                        use_bias=False,
                                        data_format=self.data_format,
                                        kernel_initializer="he_normal",
                                        kernel_regularizer=l2(
                                            self.weight_decay))
    
    self.pool1 = tf.keras.layers.MaxPooling2D(pool_size=(3, 3),
                                              strides=(2, 2),
                                              padding="same",
                                              data_format=self.data_format)
    self.batchnorm1 = tf.keras.layers.BatchNormalization(axis=axis)

    self.batchnorm2 = tf.keras.layers.BatchNormalization(axis=axis)

    # calculating the number of filters after each block
    num_filters_after_each_block = [self.num_filters]
    for i in range(1, self.num_of_blocks):
      temp_num_filters = num_filters_after_each_block[i-1] + (
          self.growth_rate * self.num_layers_in_each_block[i-1])
      # using compression to reduce the number of inputs to the
      # transition block
      temp_num_filters = int(temp_num_filters * compression)
      num_filters_after_each_block.append(temp_num_filters)

    # dense block initialization
    self.dense_blocks = []
    self.transition_blocks = []
    for i in range(self.num_of_blocks):
      self.dense_blocks.append(DenseBlock(self.num_layers_in_each_block[i],
                                          self.growth_rate,
                                          self.data_format,
                                          self.weight_decay,
                                          self.dropout_rate,
                                          self.use_se))
      if i+1 < self.num_of_blocks:
        self.transition_blocks.append(
            TransitionBlock(num_filters_after_each_block[i+1],
                            self.data_format,
                            self.weight_decay,
                            self.dropout_rate))

    # last pooling and fc layer
    if self.include_top:
      self.last_pool = tf.keras.layers.GlobalAveragePooling2D(
          data_format=self.data_format)
      self.classifier = tf.keras.layers.Dense(self.output_classes)

  def call(self, x, training=True):
    output = self.conv1(x)
    output = self.batchnorm1(output, training=training)
    output = tf.nn.relu(output)
    output = self.pool1(output)

    for i in range(self.num_of_blocks - 1):
      output = self.dense_blocks[i](output, training=training)
      output = self.transition_blocks[i](output, training=training)

    output = self.dense_blocks[
        self.num_of_blocks - 1](output, training=training)
    output = self.batchnorm2(output, training=training)
    output = tf.nn.relu(output)

    if self.include_top:
      output = self.last_pool(output)
      output = self.classifier(output)

    return output

In [0]:
densenet_121 = DenseNet(growth_rate=32, output_classes=4, num_of_blocks=4, 
                        num_layers_in_each_block=[6, 12, 24, 16])

densenet_169 = DenseNet(growth_rate=32, output_classes=4, num_of_blocks=4, 
                        num_layers_in_each_block=[6, 12, 32, 32])

densenet_201 = DenseNet(growth_rate=32, output_classes=4, num_of_blocks=4, 
                        num_layers_in_each_block=[6, 12, 48, 32])

densenet_265 = DenseNet(growth_rate=32, output_classes=4, num_of_blocks=4, 
                        num_layers_in_each_block=[6, 12, 64, 48])

In [0]:
# se_densenet_121 = DenseNet(growth_rate=32, output_classes=4, num_of_blocks=4, 
#                         num_layers_in_each_block=[6, 12, 24, 16], use_se=True)

# se_densenet_169 = DenseNet(growth_rate=32, output_classes=4, num_of_blocks=4, 
#                         num_layers_in_each_block=[6, 12, 32, 32], use_se=True)

# se_densenet_201 = DenseNet(growth_rate=32, output_classes=4, num_of_blocks=4, 
#                         num_layers_in_each_block=[6, 12, 48, 32], use_se=True)

# se_densenet_265 = DenseNet(growth_rate=32, output_classes=4, num_of_blocks=4, 
#                         num_layers_in_each_block=[6, 12, 64, 48], use_se=True)

In [25]:
# densenet_121.build(input_shape=(None, 512, 512, 3))
# densenet_121.summary()

Model: "dense_net_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3010 (Conv2D)         multiple                  9408      
_________________________________________________________________
max_pooling2d_18 (MaxPooling multiple                  0         
_________________________________________________________________
batch_normalization_3028 (Ba multiple                  256       
_________________________________________________________________
batch_normalization_3029 (Ba multiple                  4096      
_________________________________________________________________
dense_block_66 (DenseBlock)  multiple                  338304    
_________________________________________________________________
dense_block_67 (DenseBlock)  multiple                  930048    
_________________________________________________________________
dense_block_68 (DenseBlock)  multiple                 

In [0]:
# https://github.com/tensorflow/examples/blob/master/tensorflow_examples/models/densenet/densenet.py - 참고